# Phone Time per Month Analysis

In this analysis I will be sifting through large amounts of data to determine an average value a bookkeeper within the company is on the phone for. There are two differences we will want to look at here:

Incoming Calls - Which are calls that the client calls the bookkeeper at any given time (should be much lower than outgoing)

Outgoing Calls - Which are calls that the bookkeeper makes to the client (or someone else)

Using these two values we can come up with an average representation of how long a bookkeeper is on the phone for:

Total time on phone per month = Avg Incoming call time + Avg Outgoing call time

NOTE* It is important that limitations are realized. 

1) This analysis does not account for the difference due to tax time phone calls
2) Ring Central has been known not to accurately track calls which may affect more than one bookkeeper
3) Within the RingCentral data as an extension is recycled, there is a chance that the data may not have been ported over correctly. Our calculation can only be as accurate as the data we use. This is likely a small error and wouldn't affect the bookkeepers as much as it would someone that is on the phone for longer extents.

In [293]:
import pandas as pd
import datetime

# open up the two files we need to work with, the first outlines the time that needs to be summed
# the second outlines the list that will need to be cross referenced in order to get the time per month on the phone 
all_call_log = pd.read_csv("Call Logs for CC Utilization.csv")
bench_tenure = pd.read_csv("Bench Tenure.csv")

# convert the dataframe into a dictionary of values that can be searched.
bench_tenure = bench_tenure.set_index("Name")["Tenure (Months)"].to_dict()

# sets all null values to 0
test = all_call_log.isnull()
all_call_log[test] = '0'

# filters out the two types we want to look at Incoming & Outgoing calls
type_list = (all_call_log['Direction'] == 'Incoming')
in_call_log = all_call_log[type_list]

type_list = (all_call_log['Direction'] == 'Outgoing')
out_call_log = all_call_log[type_list]



In [310]:
#Analyzing Incoming Calls

# Create a filter for entries that do not equal "Voice" in the type column
in_call_log = in_call_log[in_call_log["Type"] != "Voice" ]
# Create a filter for entries that equal "Accepted" in the Action_Result column
in_call_log = in_call_log[in_call_log["Action_Result"] == "Accepted"]

# sorts values by column Extension - this is how the calculation will be based, using extension values
in_call_log = in_call_log.sort_values(by="Extension",axis=0)
# creates a list of unique extensions based on the column extension in the dataframe
unique_extension = list(in_call_log["Extension"].unique())

# creates an empty dictionary
min_per_month_values = {}


# This method calculates the duration (in minutes) using a given date time 
# It takes a value of 01:30:15 -> The duration is 1 hour, 30 minutes, and 15 seconds
# This method then takes the 1 hour multiplies it by 60 to get minuts, adds the minute value, and adds a fraction of the seconds by dividing it by 60
# The time_calc result of 01:30:15 is 90.25 minutes
def time_calc(time_val):
    time_val = time_val.split(":")
    try:
        return float(time_val[0])*60 + float(time_val[1]) + float(time_val[2])/60
    except:
        return 0


# Within this for loop, the main calculation occurs. The total time on the phone for incoming calls is calculated
# which is then divided by the tenure time the bookkeeper was at Bench for. Giving us with a value that is 
# phone time per month.
for extension in unique_extension:
    # Creates a series of duration times that will need to be summed.
    bk_time = in_call_log[in_call_log["Extension"]==extension]["Duration"].apply(time_calc)
    # Find the tenure for this persons length at Bench
    for benchmate, tenure_val in bench_tenure.items():
        if benchmate in extension:
            tenure = tenure_val
            break
        else:
            tenure = 0
    # Might receive a division by 0 error as some tenure values are 0, so we need to use a try, except block
    try:
        min_per_month_values[extension] = bk_time.sum()/tenure
    except:
        min_per_month_values[extension] = 0

# Convert dictionary into dataframe and rename column 0 to 'PhoneTime'        
per_month_values = pd.DataFrame.from_dict(min_per_month_values, orient='index').rename(columns={0:'PhoneTime'})      

# Filter out values for all but non-zero values
per_month_values = per_month_values[per_month_values['PhoneTime'] != 0.00]

# The average phone time per month value we need to include for Incoming phone Calls
average_incoming_phone_time = per_month_values['PhoneTime'].mean()

print(average_incoming_phone_time)


35.89903505393651


In [311]:
# Let's now analyze outgoing phone time
# This process is essentially the same, just using the Outgoing call data instead!

# Create a filter for entries that do not equal "Voice" in the type column
out_call_logs = out_call_log[out_call_log["Type"] == "Voice" ]


# sorts values by column Extension - this is how the calculation will be based, using extension values
out_call_logs = out_call_logs.sort_values(by="Extension",axis=0)
# creates a list of unique extensions based on the column extension in the dataframe
unique_extension = list(out_call_logs["Extension"].unique())

# creates an empty dictionary
min_per_month_values = {}


# Within this for loop, the main calculation occurs. The total time on the phone for incoming calls is calculated
# which is then divided by the tenure time the bookkeeper was at Bench for. Giving us with a value that is 
# phone time per month.
for extension in unique_extension:
    # Creates a series of duration times that will need to be summed.
    bk_time = out_call_logs[out_call_logs["Extension"]==extension]["Duration"].apply(time_calc)
    # Find the tenure for this persons length at Bench
    for benchmate, tenure_val in bench_tenure.items():
        if benchmate in extension:
            tenure = tenure_val
            break
        else:
            tenure = 0
    # Might receive a division by 0 error as some tenure values are 0, so we need to use a try, except block
    try:
        min_per_month_values[extension] = bk_time.sum()/tenure
    except:
        min_per_month_values[extension] = 0

        
# Convert dictionary into dataframe and rename column 0 to 'PhoneTime'        
per_month_values = pd.DataFrame.from_dict(min_per_month_values, orient='index').rename(columns={0:'PhoneTime'})      

# Filter out values for all but non-zero values
per_month_values = per_month_values[per_month_values['PhoneTime'] != 0.00]

# The average phone time per month value we need to include for Incoming phone Calls
average_outgoing_phone_time = per_month_values['PhoneTime'].mean()

print(average_outgoing_phone_time)

212.5382571906727


Using this method, I have found that the average time for bookkeepers on the phone:

Outgoing = 212.54 minutes
Incoming = 35.90 minutes

Total = 248.44 minutes/month

Analysis complete!